## Exploração dos dados - Novos dados de Falha

Nosso parceiro disponibilizou uma nova tabela de falhas, e iremos analisá-la nesse notebook.

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import openpyxl

In [8]:
# Intanciando o dataframe e arrumando os problemas de index

df_falhas = pd.read_csv('../data/FALHAS_PREDICT_1.csv')
df_falhas.head()


/tmp/ipykernel_10100/3532241096.py:3: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df_falhas = pd.read_csv('../data/FALHAS_PREDICT_1.csv')


,KNR,MODELO,COR,MOTOR,ESTACAO,USUARIO,HALLE,FALHA,S_GROUP_ID,DATA
0,2023-2056234,T-CROSS,2R2R,DHS,HCEQS506,4013334,ZP7,PARALAMA DIANTEIRO 01. AMASSADO,5,2024-05-20 21:31:51.992
1,2023-2056234,T-CROSS,2R2R,DHS,HCEQS506,4013334,ZP7,PORTA TRASEIRA 04. CAROÇO / PICO,4,2024-05-20 21:30:22.772
2,2023-2056234,T-CROSS,2R2R,DHS,PCEQS100,4004554,ZP5,PORTA DIANTEIRA SOLDA - RESPINGO,4,2023-12-06 06:48:29.543
3,2023-2056234,T-CROSS,2R2R,DHS,PCEQS114,2020297,ZP5,CINTO DE SEGURANÇA - FIXAÇÃO ASSOALHO ROSCA NOK,2,2023-12-05 21:36:33.284
4,2023-2056234,T-CROSS,2R2R,DHS,PCEQS620,2058570,ZP6,MÓDULO DO ACC (13) PEÇA INCORRETA ¨,9830946,2024-05-20 20:04:55.534


In [9]:
df_falhas.describe()

,KNR,MODELO,COR,MOTOR,ESTACAO,USUARIO,HALLE,FALHA,S_GROUP_ID,DATA
count,758916,758916,758916,758916,758906,758906,757311,758916,758916,758916
unique,87407,1,13,5,80,454,30,6581,19,585194
top,2024-2816497,T-CROSS,K2K2,DHS,HCEQS506,2039346,ZP7,Painel lateral 05. Sujeira na pintura,2,2023-10-05 14:27:31.000
freq,100,758916,197803,463239,381564,36311,306878,25630,231272,14


É possível notar que há dados nulos em algumas colunas, a julgar pela contagem

In [10]:
#Excluir linhas que possuem dados nulos
df_falhas = df_falhas.dropna()
df_falhas.describe()


,KNR,MODELO,COR,MOTOR,ESTACAO,USUARIO,HALLE,FALHA,S_GROUP_ID,DATA
count,757311,757311,757311,757311,757311,757311,757311,757311,757311,757311
unique,87405,1,13,5,78,451,30,6575,19,583929
top,2024-2816497,T-CROSS,K2K2,DHS,HCEQS506,2039346,ZP7,Painel lateral 05. Sujeira na pintura,2,2023-10-05 14:27:31.000
freq,100,757311,197462,462316,381564,36311,306878,25536,230807,14


In [11]:
# Ver o tipo das colunas e verificar se a retirada de dados nulos deu certo.
df_falhas.info()

<class 'pandas.core.frame.DataFrame'>
Index: 757311 entries, 0 to 758915
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   KNR         757311 non-null  object
 1   MODELO      757311 non-null  object
 2   COR         757311 non-null  object
 3   MOTOR       757311 non-null  object
 4   ESTACAO     757311 non-null  object
 5   USUARIO     757311 non-null  object
 6   HALLE       757311 non-null  object
 7   FALHA       757311 non-null  object
 8   S_GROUP_ID  757311 non-null  object
 9   DATA        757311 non-null  object
dtypes: object(10)
memory usage: 63.6+ MB


In [12]:
#Checando quantos valores únicos de tipos de falha possuímos
df_falhas["S_GROUP_ID"].unique()

array(['5', '4', '2', '9830946', '1', '-2', '133', '140', '#MULTIVALUE',
       '137', 2, 4, 133, 1, 5, 9830946, -2, 140, 137], dtype=object)

## Percepções

Os dados não estão padronizados, ou seja, há o tipo 1 numérico e o tipo 1 string, então é necessário realizar essa padronização. Além disso, há a presença de Multivalues, que não sabemos a quais categorias pertencem.

In [13]:
print(df_falhas['S_GROUP_ID'].value_counts())

S_GROUP_ID
2              230807
4              129372
2               87828
1               83836
5               82127
4               44415
5               29709
1               27925
-2              25487
9830946          4213
133              3934
-2               3790
9830946          1405
133               914
140               895
140               409
137               122
137                76
#MULTIVALUE        47
Name: count, dtype: int64


Visto que há pouquíssimos registro de Multivalue em relação ao todo (apenas 47 ocorrências), é interessante remover os dados pois há uma falta de definição do significado dele. Isso faz com que evite acontecer erros no modelo, permitindo uma melhor precisão nele.

In [14]:
#Criando um novo DataFrame retirando as ocorrências que possuem #MULTIVALULES
df_tratado = df_falhas.drop(df_falhas[df_falhas['S_GROUP_ID'] == '#MULTIVALUE'].index)
df_tratado['S_GROUP_ID'] = df_tratado['S_GROUP_ID'].astype(int)
df_tratado["S_GROUP_ID"].unique()

array([      5,       4,       2, 9830946,       1,      -2,     133,
           140,     137])

In [15]:
print(df_tratado.dtypes)

KNR           object
MODELO        object
COR           object
MOTOR         object
ESTACAO       object
USUARIO       object
HALLE         object
FALHA         object
S_GROUP_ID     int64
DATA          object
dtype: object


## Exportação

Após o tratamento dos dados, vamos exportar em csv a tabela criada, a fim de usá-la em outros notebooks.

In [16]:
df_tratado.to_csv('../data/Tabela_Falhas_Atualizada.csv', index=False)
df_atualizado = pd.read_csv('../data/Tabela_Falhas_Atualizada.csv')

In [18]:
#Visualização da tabela pós modificações
df_atualizado.head()

,KNR,MODELO,COR,MOTOR,ESTACAO,USUARIO,HALLE,FALHA,S_GROUP_ID,DATA
0,2023-2056234,T-CROSS,2R2R,DHS,HCEQS506,4013334,ZP7,PARALAMA DIANTEIRO 01. AMASSADO,5,2024-05-20 21:31:51.992
1,2023-2056234,T-CROSS,2R2R,DHS,HCEQS506,4013334,ZP7,PORTA TRASEIRA 04. CAROÇO / PICO,4,2024-05-20 21:30:22.772
2,2023-2056234,T-CROSS,2R2R,DHS,PCEQS100,4004554,ZP5,PORTA DIANTEIRA SOLDA - RESPINGO,4,2023-12-06 06:48:29.543
3,2023-2056234,T-CROSS,2R2R,DHS,PCEQS114,2020297,ZP5,CINTO DE SEGURANÇA - FIXAÇÃO ASSOALHO ROSCA NOK,2,2023-12-05 21:36:33.284
4,2023-2056234,T-CROSS,2R2R,DHS,PCEQS620,2058570,ZP6,MÓDULO DO ACC (13) PEÇA INCORRETA ¨,9830946,2024-05-20 20:04:55.534
